In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import re
from dateutil.parser import parse
import string
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import missingno as msno
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import scipy
import researchpy as rp
from matplotlib.offsetbox import AnchoredText
import pacmap
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px
import pickle
import pmdarima as pm
from ipywidgets import Button, HBox, VBox
from ipywidgets import interactive
import ipywidgets
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 
import xgboost as xgb
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

In [2]:
# Select first of vector of atollos cases (0) and append into DF:
def inicio_perdida_tph(DF, variable):
       first_case_atollo = [0]
       for x in range(1, len(DF)):
              if((DF.loc[DF.index[x], variable] == 1) & (DF.loc[DF.index[x-1], variable] == 0)):
                     first_case_atollo.append(1)
              else:
                     first_case_atollo.append(0)
       return first_case_atollo
       
# Select last of vector of atollos cases (0) and append into DF:
def final_perdida_tph(DF, variable):
       last_case_atollo = []
       for x in range((len(DF)-1)):
              if((DF.loc[DF.index[x], variable] == 1) & (DF.loc[DF.index[x+1], variable] == 0)):
                     last_case_atollo.append(1)
              else:
                     last_case_atollo.append(0)
       last_case_atollo.append(1)              
       return last_case_atollo

# Create function Atollo_mod: 0 when initial atollo; 2 when not initial atollo; 1 when no atollo:
def loss_tph(DF,variable):
    
    DF['start loss tph']=inicio_perdida_tph(DF=DF, variable=variable)
    DF['end loss tph']=final_perdida_tph(DF=DF, variable=variable)
    #Create Atollo_mod:
    variable_mod = str(variable+'_mod')    
    DF[variable_mod] =  DF[variable].replace(1,2)   
    DF.loc[DF['start loss tph']==1,variable_mod]=1
    return DF

In [3]:
# Se carga clasificador de mineralogia
kmeans = pickle.load(open('..//..//models//pickles//Clasificador_dispatch_n_4.pkl', 'rb'))
#kmeans.feature_names_in_
# Se carga la curva de celda de carga v/s granulometria (dado un subcontexto)
dicK = pickle.load(open('..//..//models//pickles//Curvas de celda de carga vs granulometria productivo actualizado.pkl', 'rb'))

# Recomendación 
def recommendationcc(granulometria,edad_sag,cluster):
 
    # Generación del subcontexto
    subcontexto = str(int(edad_sag)) + str(int(cluster))
   
    # Obtención de la recomendación
    consejo=dicK[subcontexto].loc[granulometria]["cc"]
    
    return consejo #pd.DataFrame({"recommendation":list(consejo)})

In [4]:
# read
tags=pd.read_excel("../../data/meta data/tags relevantes.xlsx")
tags_cc=tags.tag.to_list()
dic={}
for i,j in zip(tags.tag,tags.description):
    dic[i]=j

In [5]:
# Se lee csv
liners_age=pd.read_csv('../../data/processed data/liners_age_30September2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')
# Se lee csv
cleaned=pd.read_csv('../../data/processed data/cleaned_28September2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')
cleaned.sort_index(inplace=True)
cleaned.rename(columns=dic,inplace=True)
df_cleaned_age=cleaned.join(liners_age).dropna()
df_cleaned_age["dif_TPH_HH"]=df_cleaned_age["HH TPH"]-df_cleaned_age["TPH"]
df_cleaned_age["dif_CC_HH"]=df_cleaned_age["HH charge cell"]-df_cleaned_age["charge cell"]
df_cleaned_age["dif_CC_LL"]=df_cleaned_age["charge cell"]-df_cleaned_age["LL charge cell"]
df_cleaned_age["loss of TPH"]=df_cleaned_age["dif_TPH_HH"].apply(lambda x: 1 if x>100 else 0)

In [6]:
# Predict cluster
df_cleaned_age["cluster"]=kmeans.predict(df_cleaned_age[['crusher index',
                                              'sag power index',
                                               'ball work index',
                                              'chalcopyrite law',
                                              'pyrite law',
                                              'chalcocite law',
                                              'bornite law',
                                              'covelin law']])

In [7]:
# Recomendación 
Rec_HH_CC=df_cleaned_age.apply(lambda row: recommendationcc(
                                              row["granulometry"],
                                            row["Edad"],
                                            row["cluster"]
                                                    ),axis=1)

# Columna de recomendación
df_cleaned_age["recommendation"]=Rec_HH_CC

In [8]:
# Información de perdida de TPH en los proximos 5 minutos
for i in range(1,6):
    df_cleaned_age[f"loss of TPH_{i}"]=df_cleaned_age["loss of TPH"].shift(i)

df_cleaned_age["greater than HH CC"]=df_cleaned_age["dif_CC_HH"].apply(lambda x: 0 if x>1 else 1)
for i in range(1,6):
    df_cleaned_age[f"greater than HH CC_{i}"]=df_cleaned_age["greater than HH CC"].shift(i)

df_cleaned_age["less than LL CC"]=df_cleaned_age["dif_CC_LL"].apply(lambda x: 0 if x>1 else 1)
for i in range(1,6):
    df_cleaned_age[f"less than LL CC_{i}"]=df_cleaned_age["less than LL CC"].shift(i)

df_cleaned_age["greater than Rec HH CC"]=df_cleaned_age.apply(lambda x: 1 if x["recommendation"]<=x["charge cell"] else 0,axis=1)
for i in range(1,6):
    df_cleaned_age[f"greater than Rec HH CC_{i}"]=df_cleaned_age["greater than Rec HH CC"].shift(i)

df_cleaned_age["Rec HH CC less than HH CC"]=df_cleaned_age.apply(lambda x: 1 if x["recommendation"]<x["HH charge cell"] else 0,axis=1)
for i in range(1,6):
    df_cleaned_age[f"Rec HH CC less than HH CC_{i}"]=df_cleaned_age["Rec HH CC less than HH CC"].shift(i)

In [9]:
df_cleaned_age.dropna(inplace=True)
df_cleaned_age["loss of TPH"]=df_cleaned_age.apply(lambda x: 1 if (x['loss of TPH']==1) or (x['loss of TPH_1']==1) or (x['loss of TPH_2']==1) or (x['loss of TPH_3']==1) or (x['loss of TPH_4']==1) or (x['loss of TPH_5']==1) else 0,axis=1)
#df_cleaned_age["greater than HH CC"]=df_cleaned_age.apply(lambda x: 1 if (x['greater than HH CC_1']==1) or (x['greater than HH CC_2']==1) or (x['greater than HH CC_3']==1) or (x['greater than HH CC_4']==1) or (x['greater than HH CC_5']==1) else 0,axis=1)
#df_cleaned_age["less than LL CC"]=df_cleaned_age.apply(lambda x: 1 if (x['less than LL CC_1']==1) or (x['less than LL CC_2']==1) or (x['less than LL CC_3']==1) or (x['less than LL CC_4']==1) or (x['less than LL CC_5']==1) else 0,axis=1)
#df_cleaned_age["greater than Rec HH CC"]=df_cleaned_age.apply(lambda x: 1 if (x['greater than Rec HH CC_1']==1) or (x['greater than Rec HH CC_2']==1) or (x['greater than Rec HH CC_3']==1) or (x['greater than Rec HH CC_4']==1) or (x['greater than Rec HH CC_5']==1) else 0,axis=1)
#df_cleaned_age["Rec HH CC less than HH CC"]=df_cleaned_age.apply(lambda x: 1 if (x['Rec HH CC less than HH CC_1']==1) or (x['Rec HH CC less than HH CC_2']==1) or (x['Rec HH CC less than HH CC_3']==1) or (x['Rec HH CC less than HH CC_4']==1) or (x['Rec HH CC less than HH CC_5']==1) else 0,axis=1)
df_cleaned_age=loss_tph(df_cleaned_age,"loss of TPH")
df_cleaned_age.head(1)

,sag mill expert system,Limitado por Límite Alto de Tonelaje,Optimizando tonelaje,Limitado por Peso,Limitado por Corriente Bobina,Limitado por torque espesador 2,Limitado por torque espesador 3,Limitado por Potencia Bolas 1,Limitado por Potencia Bolas 2,Limitado por Pebbles,Limitado por Corriente harnero,Limitado por Potencia SAG,Limitado por limite bajo de tonelaje,Limitado por Temperatura bobina,Limitado por presión espesador 2,Limitado por presión espesador 3,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,torque,charge cell,power,TPH,solid percentage,water,speed,solid,LL charge cell,HH charge cell,HH TPH,stator current,granulometry,Edad,dif_TPH_HH,dif_CC_HH,dif_CC_LL,loss of TPH,cluster,recommendation,loss of TPH_1,loss of TPH_2,loss of TPH_3,loss of TPH_4,loss of TPH_5,greater than HH CC,greater than HH CC_1,greater than HH CC_2,greater than HH CC_3,greater than HH CC_4,greater than HH CC_5,less than LL CC,less than LL CC_1,less than LL CC_2,less than LL CC_3,less than LL CC_4,less than LL CC_5,greater than Rec HH CC,greater than Rec HH CC_1,greater than Rec HH CC_2,greater than Rec HH CC_3,greater than Rec HH CC_4,greater than Rec HH CC_5,Rec HH CC less than HH CC,Rec HH CC less than HH CC_1,Rec HH CC less than HH CC_2,Rec HH CC less than HH CC_3,Rec HH CC less than HH CC_4,Rec HH CC less than HH CC_5,start loss tph,end loss tph,loss of TPH_mod
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-10 04:05:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.76,0.5,0.02,19.7649,98.5569,15.4504,0.05,64.384706,809.23733,22008.766259,4317.030837,73.888038,1539.203093,9.063024,73.155012,770.0,850.0,3950.0,3755.398344,48.829422,1.0,-367.030837,40.76267,39.23733,0,2,892.044136,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0


In [11]:
# Se guarda en csv
df_cleaned_age.to_csv('../../data/consolidated data/df_full_'+pd.to_datetime('today').strftime('%d%B%Y')+'.csv', index=True)

In [25]:
# read df full csv
df_full=pd.read_csv('../../data/consolidated data/df_full_01November2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')
df_full.head(1)

,sag mill expert system,Limitado por Límite Alto de Tonelaje,Optimizando tonelaje,Limitado por Peso,Limitado por Corriente Bobina,Limitado por torque espesador 2,Limitado por torque espesador 3,Limitado por Potencia Bolas 1,Limitado por Potencia Bolas 2,Limitado por Pebbles,Limitado por Corriente harnero,Limitado por Potencia SAG,Limitado por limite bajo de tonelaje,Limitado por Temperatura bobina,Limitado por presión espesador 2,Limitado por presión espesador 3,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,torque,charge cell,power,TPH,solid percentage,water,speed,solid,LL charge cell,HH charge cell,HH TPH,stator current,granulometry,Edad,dif_TPH_HH,dif_CC_HH,dif_CC_LL,loss of TPH,cluster,recommendation,loss of TPH_1,loss of TPH_2,loss of TPH_3,loss of TPH_4,loss of TPH_5,greater than HH CC,greater than HH CC_1,greater than HH CC_2,greater than HH CC_3,greater than HH CC_4,greater than HH CC_5,less than LL CC,less than LL CC_1,less than LL CC_2,less than LL CC_3,less than LL CC_4,less than LL CC_5,greater than Rec HH CC,greater than Rec HH CC_1,greater than Rec HH CC_2,greater than Rec HH CC_3,greater than Rec HH CC_4,greater than Rec HH CC_5,Rec HH CC less than HH CC,Rec HH CC less than HH CC_1,Rec HH CC less than HH CC_2,Rec HH CC less than HH CC_3,Rec HH CC less than HH CC_4,Rec HH CC less than HH CC_5,start loss tph,end loss tph,loss of TPH_mod
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-10 04:05:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.76,0.5,0.02,19.7649,98.5569,15.4504,0.05,64.384706,809.23733,22008.766259,4317.030837,73.888038,1539.203093,9.063024,73.155012,770.0,850.0,3950.0,3755.398344,48.829422,1.0,-367.030837,40.76267,39.23733,0,2,892.044136,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0


In [26]:
len(df_full[df_full["start loss tph"]==1].index)

4744

In [27]:
list_df=[]
for i in df_full[df_full["start loss tph"]==1].index:
    try:
        #i="2022-02-27 12:31:00"
        inicio_perdida=i
        test=df_full[inicio_perdida:inicio_perdida+timedelta(hours=5)]
        fin_perdida=min(test[(test["end loss tph"]==1)].index)+timedelta(minutes=1)
        inicio_evento=inicio_perdida-timedelta(minutes=5)
        data=df_full.copy()[inicio_evento:fin_perdida]
        #display(data)
        dic_df={"inicio evento":inicio_evento,"inicio perdida":inicio_perdida,"fin perdida":fin_perdida,"variación HH TPH":data["HH TPH"].std()!=0,"Actua sistema de control":any(data["greater than HH CC"]==1),
        "Supera recomendación del modelo":any(data["greater than Rec HH CC"]==1),"Recomendación menor a HH CC":any(data["Rec HH CC less than HH CC"]==1),"CC menor a LL CC":any(data["less than LL CC"]==1),
        "Causalidad":any(data[data["greater than Rec HH CC"]==1].index<=inicio_perdida)}
        list_df.append(pd.DataFrame(dic_df,index=[1]))  
    except:
        #print(i)
        pass

In [15]:
df_tph_loss=pd.concat(list_df).reset_index(drop=True)
df_tph_loss.head()

,inicio evento,inicio perdida,fin perdida,variación HH TPH,Actua sistema de control,Supera recomendación del modelo,Recomendación menor a HH CC,CC menor a LL CC,Causalidad
0,2020-01-10 05:23:00,2020-01-10 05:28:00,2020-01-10 05:35:00,True,False,False,False,False,False
1,2020-01-10 07:05:00,2020-01-10 07:10:00,2020-01-10 08:41:00,False,True,False,False,False,False
2,2020-01-10 09:05:00,2020-01-10 09:10:00,2020-01-10 10:16:00,False,True,False,False,False,False
3,2020-01-10 10:15:00,2020-01-10 10:20:00,2020-01-10 11:08:00,False,True,True,True,False,False
4,2020-01-14 03:54:00,2020-01-14 03:59:00,2020-01-14 04:31:00,False,True,False,False,False,False


In [24]:
# Se guarda en csv
df_tph_loss.to_csv('df_tph_loss.csv', index=False)

In [20]:
titulos=tuple(["TPH & HH TPH", "CC, recommendation HH CC, HH CC & LL CC","granulometry","SPI","loss of TPH","diff. HH TPH between TPH",
"greater than HH CC","greater than Rec HH CC","Rec HH CC less than HH CC","less than LL CC","start loss tph","end loss tph","water","speed","solid percentage"])
fig = make_subplots(
    rows=5, cols=3,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     #shared_xaxes=True
    )


fig.add_trace(go.Scatter(x=data.index, y=data['TPH'], ##FF6511
                    mode='lines',
                    name="TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH TPH'], ##FF6511
                    mode='lines',
                    name="HH TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

#fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"], ##FF6511
#                    mode='lines',
#                    name="regressor model TPH",line=dict(width=1,color="black" ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=1, col=2)  



fig.add_trace(go.Scatter(x=data.index, y=data['LL charge cell'], ##FF6511
                    mode='lines',
                    name="LL charge cell",line=dict(width=3,),legendgroup = '1'),row=1, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH charge cell'], ##FF6511
                    mode='lines',
                    name="HH charge cell",line=dict(width=3,),legendgroup = '1'),row=1, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data["recommendation"], ##FF6511
                    mode='lines',
                    name="recommendation HH CC",line=dict(width=1,color="black"),legendgroup = '1'),row=1, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,),legendgroup = '1'),row=1, col=3)  


fig.add_trace(go.Scatter(x=data.index, y=data['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, ),legendgroup = '1'),row=5, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=5, col=3)  


for i in range(1,6):
   for j in range(1,4):
        fig.update_xaxes(tickformat="%H:%M",row=i, col=j)


fig.add_trace(go.Scatter(x=data.index, y=data[ 'water'], ##FF6511
                    mode='lines',
                    name= 'water',line=dict(width=3, ),legendgroup = '1'),row=5, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data[  'loss of TPH'], ##FF6511
                    mode='lines',
                    name=  'loss of TPH',line=dict(width=3, ),legendgroup = '1'),row=2, col=2)  


fig.add_trace(go.Scatter(x=data.index, y=data[ "dif_TPH_HH"], ##FF6511
                    mode='lines',
                    name=  "dif_TPH_HH",line=dict(width=3, ),legendgroup = '1'),row=2, col=3)  


fig.add_trace(go.Scatter(x=data.index, y=data[ 'greater than HH CC'], ##FF6511
                    mode='lines',
                    name=  'greater than HH CC',line=dict(width=3, ),legendgroup = '1'),row=3, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data[ 'less than LL CC'], ##FF6511
                    mode='lines',
                    name=  'less than LL CC',line=dict(width=3, ),legendgroup = '1'),row=4, col=1)  


fig.add_trace(go.Scatter(x=data.index, y=data[ 'start loss tph'], ##FF6511
                    mode='lines',
                    name=  'start loss tph',line=dict(width=3, ),legendgroup = '1'),row=4, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data[ 'end loss tph'], ##FF6511
                    mode='lines',
                    name=  'end loss tph',line=dict(width=3, ),legendgroup = '1'),row=4, col=3)  

fig.add_trace(go.Scatter(x=data.index, y=data[ 'greater than Rec HH CC'], ##FF6511
                    mode='lines',
                    name=  'greater than rec HH CC',line=dict(width=3, ),legendgroup = '1'),row=3, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data[ "Rec HH CC less than HH CC"], ##FF6511
                    mode='lines',
                    name=  "Rec HH CC less than HH CC",line=dict(width=3, ),legendgroup = '1'),row=3, col=3)  
fig.update_layout(height=1000, width=1500, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")
#

fig.show()

In [21]:
evento=df_tph_loss[(df_tph_loss["Actua sistema de control"]==False) & (df_tph_loss["variación HH TPH"]==False) & (df_tph_loss["CC menor a LL CC"]==False) & (df_tph_loss["Recomendación menor a HH CC"]==True) & (df_tph_loss["Supera recomendación del modelo"]==True)\
    & (df_tph_loss["Causalidad"]==True)].reset_index(drop=True)#.sample(1)
data=df_full.copy()[evento["inicio evento"].iloc[0]:evento["fin perdida"].iloc[0]]
evento

,inicio evento,inicio perdida,fin perdida,variación HH TPH,Actua sistema de control,Supera recomendación del modelo,Recomendación menor a HH CC,CC menor a LL CC,Causalidad
0,2020-03-05 11:24:00,2020-03-05 11:29:00,2020-03-05 11:44:00,False,False,True,True,False,True
1,2020-03-05 15:41:00,2020-03-05 15:46:00,2020-03-05 15:52:00,False,False,True,True,False,True
2,2020-03-10 02:19:00,2020-03-10 02:24:00,2020-03-10 02:32:00,False,False,True,True,False,True
3,2020-03-15 01:35:00,2020-03-15 01:40:00,2020-03-15 02:10:00,False,False,True,True,False,True
4,2020-03-15 13:41:00,2020-03-15 13:46:00,2020-03-15 13:54:00,False,False,True,True,False,True
...,...,...,...,...,...,...,...,...,...
226,2022-02-26 16:42:00,2022-02-26 16:47:00,2022-02-26 16:53:00,False,False,True,True,False,True
227,2022-02-27 04:51:00,2022-02-27 04:56:00,2022-02-27 05:21:00,False,False,True,True,False,True
228,2022-02-27 05:29:00,2022-02-27 05:34:00,2022-02-27 05:40:00,False,False,True,True,False,True
229,2022-02-27 05:46:00,2022-02-27 05:51:00,2022-02-27 06:48:00,False,False,True,True,False,True


In [23]:
dic_images={}
for i in range(len(evento)):
    data=df_full[evento.iloc[i]["inicio evento"]:evento.iloc[i]["fin perdida"]]
    ini_titulo=evento.iloc[i]["inicio evento"]
    #print(evento.iloc[i]["inicio evento"])
    titulos=tuple(["TPH & HH TPH", "CC, recommendation HH CC, HH CC & LL CC","granulometry","SPI","loss of TPH","diff. HH TPH between TPH",
    "greater than HH CC","greater than Rec HH CC","Rec HH CC less than HH CC","less than LL CC","start loss tph","end loss tph","water","speed","solid percentage"])
    fig = make_subplots(
        rows=5, cols=3,
        subplot_titles=titulos,
        #subplot_titles=tuple("TPH y HH TPH,"),
        #shared_xaxes=True
        )


    fig.add_trace(go.Scatter(x=data.index, y=data['TPH'], ##FF6511
                        mode='lines',
                        name="TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

    fig.add_trace(go.Scatter(x=data.index, y=data['HH TPH'], ##FF6511
                        mode='lines',
                        name="HH TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

    #fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"], ##FF6511
    #                    mode='lines',
    #                    name="regressor model TPH",line=dict(width=1,color="black" ),legendgroup = '1'),row=1, col=1)  

    fig.add_trace(go.Scatter(x=data.index, y=data['charge cell'], 
                        mode='lines',
                        name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=1, col=2)  



    fig.add_trace(go.Scatter(x=data.index, y=data['LL charge cell'], ##FF6511
                        mode='lines',
                        name="LL charge cell",line=dict(width=3,),legendgroup = '1'),row=1, col=2)  

    fig.add_trace(go.Scatter(x=data.index, y=data['HH charge cell'], ##FF6511
                        mode='lines',
                        name="HH charge cell",line=dict(width=3,),legendgroup = '1'),row=1, col=2)  

    fig.add_trace(go.Scatter(x=data.index, y=data["recommendation"], ##FF6511
                        mode='lines',
                        name="recommendation HH CC",line=dict(width=1,color="black"),legendgroup = '1'),row=1, col=2)  

    fig.add_trace(go.Scatter(x=data.index, y=data['granulometry'], ##FF6511
                        mode='lines',
                        name='granulometry',line=dict(width=3,),legendgroup = '1'),row=1, col=3)  


    fig.add_trace(go.Scatter(x=data.index, y=data['sag power index'], ##FF6511
                        mode='lines',
                        name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)  



    fig.add_trace(go.Scatter(x=data.index, y=data['speed'], ##FF6511
                        mode='lines',
                        name="speed",line=dict(width=3, ),legendgroup = '1'),row=5, col=2)  

    fig.add_trace(go.Scatter(x=data.index, y=data['solid percentage'], ##FF6511
                        mode='lines',
                        name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=5, col=3)  


    for u in range(1,6):
        for v in range(1,4):
                fig.update_xaxes(tickformat="%H:%M",row=u, col=v)


    fig.add_trace(go.Scatter(x=data.index, y=data[ 'water'], ##FF6511
                        mode='lines',
                        name= 'water',line=dict(width=3, ),legendgroup = '1'),row=5, col=1)  



    fig.add_trace(go.Scatter(x=data.index, y=data[  'loss of TPH'], ##FF6511
                        mode='lines',
                        name=  'loss of TPH',line=dict(width=3, ),legendgroup = '1'),row=2, col=2)  


    fig.add_trace(go.Scatter(x=data.index, y=data[ "dif_TPH_HH"], ##FF6511
                        mode='lines',
                        name=  "dif_TPH_HH",line=dict(width=3, ),legendgroup = '1'),row=2, col=3)  


    fig.add_trace(go.Scatter(x=data.index, y=data[ 'greater than HH CC'], ##FF6511
                        mode='lines',
                        name=  'greater than HH CC',line=dict(width=3, ),legendgroup = '1'),row=3, col=1)  

    fig.add_trace(go.Scatter(x=data.index, y=data[ 'less than LL CC'], ##FF6511
                        mode='lines',
                        name=  'less than LL CC',line=dict(width=3, ),legendgroup = '1'),row=4, col=1)  


    fig.add_trace(go.Scatter(x=data.index, y=data[ 'start loss tph'], ##FF6511
                        mode='lines',
                        name=  'start loss tph',line=dict(width=3, ),legendgroup = '1'),row=4, col=2)  

    fig.add_trace(go.Scatter(x=data.index, y=data[ 'end loss tph'], ##FF6511
                        mode='lines',
                        name=  'end loss tph',line=dict(width=3, ),legendgroup = '1'),row=4, col=3)  

    fig.add_trace(go.Scatter(x=data.index, y=data[ 'greater than Rec HH CC'], ##FF6511
                        mode='lines',
                        name=  'greater than rec HH CC',line=dict(width=3, ),legendgroup = '1'),row=3, col=2)  

    fig.add_trace(go.Scatter(x=data.index, y=data[ "Rec HH CC less than HH CC"], ##FF6511
                        mode='lines',
                        name=  "Rec HH CC less than HH CC",line=dict(width=3, ),legendgroup = '1'),row=3, col=3)  
    fig.update_layout(height=1000, width=1500, title_text=f"Temporal signal analysis - {ini_titulo}")
    fig.update_layout(hovermode="x unified")
    #

    #fig.show()
    fecha=ini_titulo.strftime("%Y-%m-%d")
    dic_images[i]=fig
    fig.write_image(f"../../images/Escenarios perdida de TPH por pasar recomendación (good) (sin sistema de control)/{i}_fig_{fecha}.png")
    print(i)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
